In [36]:
# __author__ = John Boudreaux
class strategy:
    '''An object to contain different types of strategies for the El Farol
    problem that can be adopted by different agents. Expects a "strat_type" arg,
    which descibes the type of strategy adopted. Valid strategy options include:
    "same_as_past", which picks a strategy from n iterations ago,
    "flip", which takes the number of attendees at week n and subtracts it
    from the total number of agents,
    "minimum_of_last", which will return the minimum attendees from
    the last n iterations,
    "avg", which takes the average of the past n weeks,
    "maximum_of_last", which will return the maximum attendees from the last
    n iterations'''

    approved_strats = ["same_as_past", "flip", "minimum_of_last", "avg", "maximum_of_last"]

    def __init__(self, strat_type, n):
        if strat_type not in strategy.approved_strats:
            raise ValueError("Please pick an approved strategy from documentation")
        else:
            self.strat_type =  strat_type
            self.n = n

    def strat_evaluate(self, num_of_agents, history, n, strat_type):
        '''A script to evaluate the strategy for a given agent and the history'''
        # note to self: I wish Python had a convenient switch/case protocol
        # all scripts here follow similar order: what to do if the length of the
        # history is zero, then if it is less than the required n, and then the
        # general case.

        if strat_type == "same_as_past":
            if len(history) < n:
                if len(history) == 0:
                    expected_num = 0
                else:
                    expected_num = history[0]
            else:
                expected_num = history[len(history)-n]

        elif strat_type == "flip":
            if len(history) < n:
                expected_num = num_of_agents
            else:
                expected_num = num_of_agents - history[len(history)-n]

        elif strat_type == "minimum_of_last":
            if len(history) < n:
                if len(history) == 0:
                    expected_num = 0
                else:
                    valid_search = history[0:len(history)]
                    expected_num = min(valid_search)
            else:
                valid_search = history[(len(history)-n):len(history)]
                expected_num = min(valid_search)

        elif strat_type == "maximum_of_last":
            if len(history) < n:
                if len(history) == 0:
                    expected_num = num_of_agents
                else:
                    valid_search = history[0:len(history)]
                    expected_num = max(valid_search)
            else:
                valid_search = history[(len(history)-n):len(history)]
                expected_num = max(valid_search)

        elif strat_type == "avg":
            if len(history) < n:
                if len(history) == 0:
                    expected_num = round(num_of_agents / 2)
                else:
                    valid_search = history[0:len(history)]
                    expected_num = round(sum(valid_search)/len(valid_search))
            else:
                valid_search = history[(len(history)-n):len(history)]
                expected_num = round(sum(valid_search)/len(valid_search))

        return(expected_num)


In [48]:
# __author__ = John Boudreaux

class agent:
    '''A class to store properties about agents for El Farol and minority
    problem games. Expects a memory length as an integer (default is 2),
    and a number of strategies as an integer'''

    def __init__(self, memory_length=3, num_strats=3):
        # need to select n at random from uniform distribution,
        # strategy at random from uniform distribution
        available_strats = strategy.approved_strats
        self.current_strat = {"index": None, "expected_val": None, "go": None}
        # python 3.6 has a more convenient random.choices() operation,
        # but I am only assuming verion >= 3.5
        self.strategies = {}
        for i in range(0, num_strats):
            picked_strat = random.choice(available_strats)
            picked_n = random.choice(range(1, memory_length+1))
            self.strategies[i] = {'strat': strategy(picked_strat, picked_n),\
                                  'n': picked_n, 'votes': 0,
                                 'strat_type': picked_strat}

    def select_and_eval_strat(self, history, cutoff_val):
        # although it may seem a bit much to have both selection and evaluation
        # in one method, it will reduce the number of temporary objects
        votes_list = [self.strategies[i]['votes'] for i in range(len(self.strategies))]
        max_votes = max(votes_list)
        max_indices = [index for index, val in enumerate(votes_list) if val == max_votes]
        # choose the strategy with the most votes. if tied, choose random
        if len(max_indices) > 1:
            which_index = random.choice(max_indices)
        else:
            which_index = max_indices[0]
        active_strat = self.strategies[which_index]
        self.current_strat['index'] = which_index
        # evaluate the selected strategy, bring back the expected val
        # provide convenient vars for readability
        local_n = self.strategies[which_index]['n']
        local_strat_type = self.strategies[which_index]['strat'].strat_type
        num_of_agents = sim.num_of_agents
        expected_val = active_strat['strat'].strat_evaluate(num_of_agents, \
                       history, n = local_n, strat_type = local_strat_type)
        self.current_strat['expected_val'] = expected_val

        if expected_val < cutoff_val:
            self.current_strat['go'] = 1
        else:
            self.current_strat['go'] = 0

    def update_votes(self, winning_num):
        #function to update the votes for strategies
        which_index = self.current_strat['index']

        if self.current_strat['go'] == winning_num:
            self.strategies[which_index]['votes'] += 1
        else:
            self.strategies[which_index]['votes'] -= 1


In [47]:
# __author__ = John Boudreaux

class simulation:
    '''A class to house all the parameters for a simulation of the El Farol
    problem, and to store its outputs'''
    # going to need to pass all parameters through here, since agents are More
    # or less a child class, and strategies are a child class of agents

    def __init__(self, num_of_agents, iterations, cutoff_val, mem_length, num_strats):
        '''Simulation object expects agents of class agent,
         and a number of iterations that is an integer'''
        self.num_of_agents = num_of_agents
        self.mem_length = mem_length
        self.target_value = math.ceil(cutoff_val * self.num_of_agents)
        self.iterations = iterations
        self.num_strats = num_strats
        self.history = []
        # create an object to contain all the agents and create them
        self.agents = {}
        for i in range(self.num_of_agents):
            self.agents[i] = agent(memory_length = self.mem_length, num_strats = self.num_strats)


    def run_simulation(self):
        '''Code to loop through and run simulation. Takes variables from
        namespace of simulation'''
        print("Beginning simulation")
        for i in range(self.iterations):
            agents_going_to_bar = 0
            for j in range(len(self.agents)):
                self.agents[j].select_and_eval_strat(history = self.history, cutoff_val = self.target_value)
                agents_going_to_bar += self.agents[j].current_strat['go']

            # if the total amount of agents at the bar is above the target value,
            # I would have had more fun staying home
            self.history.append(agents_going_to_bar)

            if agents_going_to_bar > self.target_value:
                winning_val = 0
            else:
                winning_val = 1

            for j in range(len(self.agents)):
                self.agents[j].update_votes(winning_val)

            if i % 10 == 0:
                print("Iteration", i, "of", self.iterations, "completed!")

        print("Simulation completed!")

        # loop for number of simulations
            # select the strategy and evaluate it as a go/no go
            # record total number of gos, append history. figure out whether go or no go was a win
            # if the strategy is successful, add a vote to it. if not, take away a vote
        #


In [13]:
import random
import math
# fun text from: https://www.messletters.com/en/big-text/
print(""":::::::::: :::         ::::::::::     :::     :::::::::   ::::::::  :::
:+:        :+:         :+:          :+: :+:   :+:    :+: :+:    :+: :+:
+:+        +:+         +:+         +:+   +:+  +:+    +:+ +:+    +:+ +:+
+#++:++#   +#+         :#::+::#   +#++:++#++: +#++:++#:  +#+    +:+ +#+
+#+        +#+         +#+        +#+     +#+ +#+    +#+ +#+    +#+ +#+
#+#        #+#         #+#        #+#     #+# #+#    #+# #+#    #+# #+#
########## ##########  ###        ###     ### ###    ###  ########  ##########  """)
print("")
print("Let's run a simulation of the El Farol problem. I'll need a few parameters...")
print("")

:::::::::: :::         ::::::::::     :::     :::::::::   ::::::::  :::
:+:        :+:         :+:          :+: :+:   :+:    :+: :+:    :+: :+:
+:+        +:+         +:+         +:+   +:+  +:+    +:+ +:+    +:+ +:+
+#++:++#   +#+         :#::+::#   +#++:++#++: +#++:++#:  +#+    +:+ +#+
+#+        +#+         +#+        +#+     +#+ +#+    +#+ +#+    +#+ +#+
#+#        #+#         #+#        #+#     #+# #+#    #+# #+#    #+# #+#
########## ##########  ###        ###     ### ###    ###  ########  ##########  

Let's run a simulation of the El Farol problem. I'll need a few parameters...



In [50]:
# user input for parameter selection
iterations = int(input("Select a number of iterations for your simulation: "))
num_of_agents = int(input("Select a number of agents for your simulation: "))
mem_length = int(input("Select a number for the length of the agent memories: "))
num_strats = int(input("Select a number of strategies for each agent: "))
cutoff = int(input("Select a cutoff percentage (integer value): ")) / 100
seed = int(input("Choose a seed value (integer value): "))

print("")

Select a number of iterations for your simulation: 10000
Select a number of agents for your simulation: 500
Select a number for the length of the agent memories: 10
Select a number of strategies for each agent: 3
Select a cutoff percentage (integer value): 60
Choose a seed value (integer value): 45



In [51]:
sim = simulation(num_of_agents, iterations, cutoff, mem_length, num_strats)
sim.run_simulation()

print(sim.history)

Beginning simulation
Iteration 0 of 10000 completed!
Iteration 10 of 10000 completed!
Iteration 20 of 10000 completed!
Iteration 30 of 10000 completed!
Iteration 40 of 10000 completed!
Iteration 50 of 10000 completed!
Iteration 60 of 10000 completed!
Iteration 70 of 10000 completed!
Iteration 80 of 10000 completed!
Iteration 90 of 10000 completed!
Iteration 100 of 10000 completed!
Iteration 110 of 10000 completed!
Iteration 120 of 10000 completed!
Iteration 130 of 10000 completed!
Iteration 140 of 10000 completed!
Iteration 150 of 10000 completed!
Iteration 160 of 10000 completed!
Iteration 170 of 10000 completed!
Iteration 180 of 10000 completed!
Iteration 190 of 10000 completed!
Iteration 200 of 10000 completed!
Iteration 210 of 10000 completed!
Iteration 220 of 10000 completed!
Iteration 230 of 10000 completed!
Iteration 240 of 10000 completed!
Iteration 250 of 10000 completed!
Iteration 260 of 10000 completed!
Iteration 270 of 10000 completed!
Iteration 280 of 10000 completed!
Iter

Iteration 3190 of 10000 completed!
Iteration 3200 of 10000 completed!
Iteration 3210 of 10000 completed!
Iteration 3220 of 10000 completed!
Iteration 3230 of 10000 completed!
Iteration 3240 of 10000 completed!
Iteration 3250 of 10000 completed!
Iteration 3260 of 10000 completed!
Iteration 3270 of 10000 completed!
Iteration 3280 of 10000 completed!
Iteration 3290 of 10000 completed!
Iteration 3300 of 10000 completed!
Iteration 3310 of 10000 completed!
Iteration 3320 of 10000 completed!
Iteration 3330 of 10000 completed!
Iteration 3340 of 10000 completed!
Iteration 3350 of 10000 completed!
Iteration 3360 of 10000 completed!
Iteration 3370 of 10000 completed!
Iteration 3380 of 10000 completed!
Iteration 3390 of 10000 completed!
Iteration 3400 of 10000 completed!
Iteration 3410 of 10000 completed!
Iteration 3420 of 10000 completed!
Iteration 3430 of 10000 completed!
Iteration 3440 of 10000 completed!
Iteration 3450 of 10000 completed!
Iteration 3460 of 10000 completed!
Iteration 3470 of 10

Iteration 6120 of 10000 completed!
Iteration 6130 of 10000 completed!
Iteration 6140 of 10000 completed!
Iteration 6150 of 10000 completed!
Iteration 6160 of 10000 completed!
Iteration 6170 of 10000 completed!
Iteration 6180 of 10000 completed!
Iteration 6190 of 10000 completed!
Iteration 6200 of 10000 completed!
Iteration 6210 of 10000 completed!
Iteration 6220 of 10000 completed!
Iteration 6230 of 10000 completed!
Iteration 6240 of 10000 completed!
Iteration 6250 of 10000 completed!
Iteration 6260 of 10000 completed!
Iteration 6270 of 10000 completed!
Iteration 6280 of 10000 completed!
Iteration 6290 of 10000 completed!
Iteration 6300 of 10000 completed!
Iteration 6310 of 10000 completed!
Iteration 6320 of 10000 completed!
Iteration 6330 of 10000 completed!
Iteration 6340 of 10000 completed!
Iteration 6350 of 10000 completed!
Iteration 6360 of 10000 completed!
Iteration 6370 of 10000 completed!
Iteration 6380 of 10000 completed!
Iteration 6390 of 10000 completed!
Iteration 6400 of 10

In [28]:
sample_list = [0, 3, 1, 2]
which_index = 2
sample_list[which_index]

1